# Hooks

> Monitoring DL models statistics along the way ...

In [ ]:
#| default_exp hooks

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
# from copy import copy
# import math
# import fastcore.all as fc
# from fastprogress import progress_bar, master_bar
# from operator import attrgetter
# from collections.abc import Mapping

# import matplotlib.pyplot as plt

# import torch
# from torchvision import transforms as T
# from torcheval.metrics import Mean

In [ ]:
#| export
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

from lssm.loading import load_ossl
from lssm.models import conv
from lssm.callbacks import to_cpu
from lssm.preprocessing import Log1p, SNV
from lssm.dataloaders import SpectralDataset, get_dls

import torch
from torch import nn, tensor

In [ ]:
#|eval: false
analytes = 'clay.tot_usda.a334_w.pct' # Let's play with Clay
spectra_type = 'mir'

data = load_ossl(analytes, spectra_type)
X, y, X_names, smp_idx, ds_name, ds_label = data

X = Pipeline([('snv', SNV())]).fit_transform(X)
y = Log1p().fit_transform(y)

Reading & selecting data ...


In [ ]:
#|eval: false
# Train/valid split
n_smp = None # For demo. purpose
X_train, X_valid, y_train, y_valid = train_test_split(X[:n_smp, :], y[:n_smp],
                                                      test_size=0.1,
                                                      stratify=ds_name[:n_smp],
                                                      random_state=41)

# Get PyTorch datasets
train_ds, valid_ds = [SpectralDataset(X, y, )
                      for X, y, in [(X_train, y_train), (X_valid, y_valid)]]

# Then PyTorch dataloaders
dls = get_dls(train_ds, valid_ds, bs=32)

In [ ]:
#| export
from functools import partial
from torch.optim import lr_scheduler

from torcheval.metrics import R2Score
from lssm.callbacks import (MetricsCB, BatchSchedCB, BatchTransformCB,
                            DeviceCB, TrainCB, ProgressCB)
from lssm.transforms import GADFTfm, _resizeTfm, StatsTfm
from lssm.learner import Learner

In [ ]:
#| export
class ToyCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            conv(1, 8, ks=5),        # 8, 851
            conv(8, 16),             # 16, 426
            conv(16, 32),            # 32, 213
            conv(32, 64),            # 64, 107
            nn.Flatten(),
            nn.Linear(64*107, 1))
        
    def forward(self, x):
        return self.layers(x)

In [ ]:
#|eval: false
epochs = 8
lr = 1e-3

tmax = epochs * len(dls.train)
sched = partial(lr_scheduler.OneCycleLR, max_lr=lr, total_steps=tmax)
xtra = [BatchSchedCB(sched)]
cbs = [DeviceCB(), TrainCB(), 
       MetricsCB(r2=R2Score()),
       ProgressCB(plot=False)]

learn = Learner(ToyCNN(), dls, nn.MSELoss(), lr=lr,
                cbs=cbs+xtra, opt_func=optim.AdamW)

In [ ]:
#|eval: false
learn.fit(epochs)

r2        loss      epoch     train                                                         
0.002     0.809     0         train     
0.739     0.213     0         eval                                                      
0.793     0.168     1         train                                                         
0.746     0.207     1         eval                                                      
0.825     0.142     2         train                                                         
0.816     0.150     2         eval                                                      
0.844     0.127     3         train                                                         
0.851     0.121     3         eval                                                      
0.855     0.117     4         train                                                         
0.856     0.117     4         eval                                                      
0.866     0.109     5         train              

In [ ]:
#| export
class Hook():
    def __init__(self, m, f): self.hook = m.register_forward_hook(partial(f, self))
    def remove(self): self.hook.remove()
    def __del__(self): self.remove()

In [ ]:
def fit(model, epochs=1, xtra_cbs=None):
    learn = Learner(model, dls, loss_func=F.cross_entropy, lr=0.6, cbs=cbs+fc.L(xtra_cbs))
    learn.fit(epochs)
    return learn

In [ ]:
def append_stats(hook, mod, inp, outp):
    if not hasattr(hook,'stats'): hook.stats = ([],[])
    acts = to_cpu(outp)
    hook.stats[0].append(acts.mean())
    hook.stats[1].append(acts.std())

In [ ]:
import nbdev; nbdev.nbdev_export()